In [1]:
import hypar as hp
import numpy as np
from scipy.optimize import fsolve

# Parameters for waves and hypar.
The wave force is obtained using Epirical Formula.

In [8]:
# Parameters that define Hypar Geometry
b1 = 4 #longitudinal dimension of lower panel
b2 = 4 #longitudinal dimension of upper panel

c1 = 4 #Left transverse dimension
c2 = 4 #Right transverse dimension

A = (b1+b2)*(c1+c2) #Projected area
r = 0.03*A #Rise, quantifies how warpped the structure is



theta_star = 0 * np.pi / 180 # angle of inclination
h_star = (b1+b2)*np.sin(theta_star)#Height

#FEM discretization
n_b1 = 12 #Number of quad elements along longitudinal direction of lower panel
n_b2 = 12 #Number of quad elements along longitudinal direction of upper panel

n_c1 = 12 #Number of quad elements along transverse direction of left panel
n_c2 = 12 #Number of quad elements along transverse direction of right panel



In [9]:
#Wave characteristics
Hw = 1 #Wave height for goda, doesn't matter here
dw = 4 #Inundation
d = 9 #Water depth
g = 9.8 #gravitational accel.
T = 6 #Wave period

def eq_k (k,d=d,g=g):
    return 2*np.pi/T - np.sqrt(g*k * np.tanh(k*d))
def wavenum_k(T,d):
    return abs(fsolve(eq_k,1))
def lbd(T,d,g):
    k = abs(fsolve(eq_k,1))
    return k,2*np.pi / k
Lw = float((lbd(T,d,g))[1][0]) #Wavelength

#Structure's position to the slope top
d_to_stop = 0
#Slope value
bot_x = 80
top_x = 116
slope = (d-dw)/(bot_x-top_x)

if (5*Hw-d_to_stop) >= (d-dw)/slope:
    hb = d
elif (5*Hw-d_to_stop) <= 0:
    hb = dw
else:
    hb =dw + (5*Hw-d_to_stop)*slope

In [10]:
Sym_coord, StaticP_Area, StaticP_val, StaticP_vec, WaveP_Area, WaveP_vec, WaveP_val,nk,Tri_A = hp.getCoordNForce(n_c1,n_c2,n_b1,n_b2,dw,theta_star,r,c1,c2,b1,b2,Hw,hb,d,Lw) #Get coordinates & nodes
Area_cnct = hp.getShellConnectivity(n_c1,n_c2,n_b1,n_b2)

/home/gaoyuanw/Github/JaxSSO/Examples/Hypar_Wave/hypar.py:254: RuntimeWarning: invalid value encountered in scalar divide
  l1_prime = l1_prime / np.sin(theta_1[i])
/home/gaoyuanw/Github/JaxSSO/Examples/Hypar_Wave/hypar.py:256: RuntimeWarning: divide by zero encountered in scalar divide
  l2_prime = l2_prime / np.sin(theta_2[i])
/home/gaoyuanw/Github/JaxSSO/Examples/Hypar_Wave/hypar.py:257: RuntimeWarning: divide by zero encountered in divide
  l1 = min(dw,hv) / np.sin(theta_1[i])
/home/gaoyuanw/Github/JaxSSO/Examples/Hypar_Wave/hypar.py:258: RuntimeWarning: divide by zero encountered in divide
  l2 = Macaulay(dw - hv) / np.sin(theta_2[i])


In [11]:
Sym_coord

array([[ 0.   ,  4.   ,  0.   ],
       [ 0.   ,  3.667, -0.   ],
       [ 0.   ,  3.333, -0.   ],
       ...,
       [ 8.   , -3.333, -0.   ],
       [ 8.   , -3.667, -0.   ],
       [ 8.   , -4.   ,  0.   ]])

# Single-instance ML v.s. Gradient Descent

Goal:
1. Find the best hypar geometry (characterized by $r$ and $\theta$,b1,b2) under waves loads (characterized by wave heights, periods, etc.)
2. Wave force: use empirical formula to approximate
3. Linear FEA: [K]{u}={f}

Why ML instead of direct gradient-based shape optimization?:
1. IDK

ML-Input (A wave scenario)
1. Wave characteristics (H,T)

ML-Output (direct)/Shape input:
1. $r$ rise of hypar
2. $\theta$ inclination angle
3. $b_1,b_2$ quantifies the longitudinal assymetry

Shape inputs determine:
1. wave forces (Goda empirical)
2. structural response (strain energy)
3. mass of structure (material)

ML-loss:
1. A weighted form of strain energy + material usage


Train a **single-instance** ML model:
Inputs (wave condition) -> ML-model -> Outputs (structural parameters) -> strain energy (Loss)

By updating ML-model and minimizing loss, the outputs are found: the best structure under such wave condition